# Explore the dataset


In this notebook, we will perform an EDA (Exploratory Data Analysis) on the processed Waymo dataset (data in the `processed` folder). In the first part, you will create a function to display 

In [2]:
from utils import get_dataset
from waymo_open_dataset import dataset_pb2 as open_dataset
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import tensorflow.compat.v1 as tf


%matplotlib inline

In [4]:
filename = "/data/waymo/*.tfrecord"
dataset = tf.data.TFRecordDataset(filename, compression_type='')

for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break

NotFoundError: /data/waymo/*.tfrecord; No such file or directory

In [24]:
def countRecords(ds:tf.data.Dataset):
  count = 0

  if tf.executing_eagerly():
    # TF v2 or v1 in eager mode
    for r in ds:
      count += 1
  else:  
    # TF v1 in non-eager mode
    iterator = tf.compat.v1.data.make_one_shot_iterator(ds)
    next_batch = iterator.get_next()
    with tf.compat.v1.Session() as sess:
      try:
        while True:
          sess.run(next_batch)
          count = count+1    
      except tf.errors.OutOfRangeError:
        pass
  
  return count

In [27]:

filenames =['/data/waymo/segment-1005081002024129653_5313_150_5333_150_with_camera_labels.tfrecord']

raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='')

print(countRecords(raw_dataset))

199


## Write a function to display an image and the bounding boxes

Implement the `display_instances` function below. This function takes a batch as an input and display an image with its corresponding bounding boxes. The only requirement is that the classes should be color coded (eg, vehicles in red, pedestrians in blue, cyclist in green).

In [4]:
def display_instances(batch):
    """
    This function takes a batch from the dataset and display the image with 
    the associated bounding boxes.
    """
    try:
        # color map
        color_map = {1: 'red', 2: 'blue', 4: 'green'}
    
    
        fig, ax = plt.subplots() 
        im = batch["image"].numpy()
        ax.imshow(im)
    
        # draw bounding boxes and add color to different class labels
        for coord, label in zip(batch["groundtruth_boxes"].numpy(), batch["groundtruth_classes"].numpy()):
            xy = (coord[1]*im.shape[1], coord[0]*im.shape[0])
            #taken from class notes
            width = (coord[3] - coord[1])*im.shape[1]
            height = (coord[2] - coord[0])*im.shape[0]
            rec = patches.Rectangle(xy, width, height, linewidth=1, edgecolor=color_map[label], facecolor='none')
            ax.add_patch(rec)
        plt.axis("off")
    except Exception as err:
        print(err)
    

## Display 10 images 

Using the dataset created in the second cell and the function you just coded, display 10 random images with the associated bounding boxes. You can use the methods `take` and `shuffle` on the dataset.

In [ ]:
plt.figure(figsize=(10, 10))
color_map = {1: 'red', 2: 'blue', 3: 'green'}
plt.axis("off")

for batch in dataset.take(1):

    picture, ax = plt.subplots()
    image = batch["image"].numpy()
    ax.imshow(image)
    

    for c, label in zip(batch["groundtruth_boxes"].numpy(), batch["groundtruth_classes"].numpy()):
        print(i)
        size = image.shape[1] * image.shape[0] / 1920
            
        # from class notes
        width = (c[3] - c[1]) * size 
        height = (c[2] - c[0]) * size
        
        rec = patches.Rectangle(xy, width, height, linewidth=1, edgecolor=color_map[label], facecolor='none')
        ax.add_patch(rec)
        i += 1
        
    plt.axis("off")

plt.show()

In [6]:
plt.figure(figsize=(10, 10))

for i in range(10):
    for batch in dataset.shuffle(75, reshuffle_each_iteration = True).take(1):
        display_instances(batch)
        
plt.show()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 40: invalid start byte

## Additional EDA

In this last part, you are free to perform any additional analysis of the dataset. What else would like to know about the data?
For example, think about data distribution. So far, you have only looked at a single file...